<h1>Home Loans Predictions and Analysis </h1>
<p>The purpose of this project is to analyze and make predictions with a dataset on home loans from 2007. Features will be analyzed with the intention of finding usefulness in the task of making predictions whether loans were defaulted on or not. After the features are analyzed they will be cleaned and engineered in such a way that they can be fed into a machine learning model. Various machine learning models will then be experimented with so as to find the one which best predicts loan-status without overfitting. Further considerations, such as how to account for the data being unbalanced (that is, the data containing much more successful loans than defaults) and whether it is better to have a more conservative model (a model which penalizes false positives more than false negatives) will also be adressed. </p>

<Importing libraries and reading in the dataset\>

In [329]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.feature_selection import RFECV
import time
%matplotlib inline

In [330]:
data = pd.read_csv('loans_2007.csv')
data.head()
##The warning isn't important, it regards a column that will end up being dropped

c:\users\danger fellner\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


<h1> Part 1: Manually dropping columns (features) that don't make sense to include </h1>

<Listing the names of each column in the dataset \>

In [331]:
for i in data.columns:
    print(i)

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
loan_status
pymnt_plan
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
inq_last_6mths
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
last_credit_pull_d
collections_12_mths_ex_med
policy_code
application_type
acc_now_delinq
chargeoff_within_12_mths
delinq_amnt
pub_rec_bankruptcies
tax_liens


<p>Of the above columns, the ones selected to be dropped in the cell below are those which either leak data about the future (and so would bias any predictions made), serve as some form of identification number that wouldn't make sense to use to make predictions with, or would require more information than present in the dataset to be useful. </p>

In [332]:
droppable_columns = ['id','member_id','funded_amnt','funded_amnt_inv','grade','sub_grade','emp_title','issue_d','zip_code','out_prncp','out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt','earliest_cr_line','last_credit_pull_d']
data = data.drop(droppable_columns,axis=1)
data.columns

Index(['loan_amnt', 'term', 'int_rate', 'installment', 'emp_length',
       'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
       'pymnt_plan', 'purpose', 'title', 'addr_state', 'dti', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util',
       'total_acc', 'initial_list_status', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies',
       'tax_liens'],
      dtype='object')

<h1> Part 2: Cleaning remaining numeric data </h1>


<p><b>Note 1: </b> "loan_status" is the name of the column regarding whether loans have been paid off or not. It is the value that will be predicted. It has however, more possible values than the simple binary "paid off or not paid off". All possible values are printed below </p>

In [333]:
data['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

<p><b>Note 2:</b> The only two values of interest for this project are "Fully Paid" and "Charged off". According to the data dictionary, the remaining categories are about ongoing loans (even "default"), whereas "Fully Paid" and "Charged off" pertain to completed loans (with "Fully paid" meaning the loan was paid and "Charged off" meaning the loan was defaulted on). </p>
<p> The code below removes all rows which don't contain either a value of "Fully  paid" or of "Charged off". It then converts the categorical data to numeric data, setting "Fully paid" to 1 and "Charged off" to 0. </p>

In [334]:
data = data[(data['loan_status'] == 'Fully Paid') | (data['loan_status'] == 'Charged Off')]
data = data.replace({'Fully Paid':1 , 'Charged Off' : 0})

<p> The code below next drops all columns which contain the same value. Such columns are useless for making predictions with </p>

In [335]:
drop_columns = []
for i in data.columns:
    if len(data[i].value_counts()) == 1:
        drop_columns.append(i)
data = data.drop(drop_columns,axis=1)
print(len(data.columns))
#23 columns remain in the dataset

21


<p><b>Note 3: </b> Now analyzing the presence of null values in the dataset</p>
<p> The code below displays each column containing null values, and how many null vaues contained in each </p>

In [336]:
null_counts =  data.isnull().sum()
print(null_counts[null_counts > 0])

emp_length              1036
title                     11
revol_util                50
pub_rec_bankruptcies     697
dtype: int64


<p>The two categories with the largest number of null values are "emp_length" and "pub_rec_bankruptcies". The code below will inspect these two categories further by printing each of their unique values and the percentage they make up the category. </p>

In [337]:
print(data['pub_rec_bankruptcies'].value_counts(normalize=True, dropna=False))
print("\n ---------------- \n")
print(data['emp_length'].value_counts(normalize=True, dropna=False))

0.0    0.939438
1.0    0.042456
NaN    0.017978
2.0    0.000129
Name: pub_rec_bankruptcies, dtype: float64

 ---------------- 

10+ years    0.220454
< 1 year     0.116766
2 years      0.111117
3 years      0.103843
4 years      0.086717
5 years      0.082770
1 year       0.082100
6 years      0.056255
7 years      0.044313
8 years      0.037245
9 years      0.031700
NaN          0.026722
Name: emp_length, dtype: float64


<p><b>Note 4: </b>95% of values in "pub_rec_bankruptcies" are either 0.0 or NaN. It is unlikely that a column with such low variance will be helpful, so it will be dropped. Next, for the time being, rows containing NaN values will be dropped. Besides "emp_length", the other columns containing NaN values hardly have any, so it's unlikely to affect the analysis by dropping them. For "emp_length", dropping rows which contain a NaN value is more of an unfortunate necessity. "Emp_length" is most likely a useful feature, but it is unclear what NaN corresponds to. For the time being, the way of dealing with such values will be dropping them. Depending on how the future analysis goes, it may make more sense to instead set such Nan values to 0 or the mean. </p>

In [338]:
data = data.drop(['pub_rec_bankruptcies'],axis=1)
data = data.dropna() #drop rows containing nan values

<h1>Part 3: Cleaning non-numeric data </h1>

<p><b>Note 1: </b> Inspecting the non-numeric data </p>

<printing the datatypes of the remaining columns and number corresponding to each\>

In [339]:
print(data.dtypes.value_counts())

float64    10
object      9
int64       1
dtype: int64


<creating a new dataframe containing just columns stored as an object (non-numeric columns)>

In [340]:
object_columns_df = data.select_dtypes(include=['object']).copy()
object_columns_df.head()

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,revol_util
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,83.7%
1,60 months,15.27%,< 1 year,RENT,Source Verified,car,bike,GA,9.4%
2,36 months,15.96%,10+ years,RENT,Not Verified,small_business,real estate business,IL,98.5%
3,36 months,13.49%,10+ years,RENT,Source Verified,other,personel,CA,21%
5,36 months,7.90%,3 years,RENT,Source Verified,wedding,My wedding loan I promise to pay back,AZ,28.3%


<p><b>Note 2: </b>Some of these columns can easily be converted to numerical data. The ones that cannot will be inspected more closely. Will first look at their unique values using the code below. </p>

In [341]:
inspect_cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state','title','purpose']
##inspect_cols being columns that have more going on than just dropping a percent sign
non_inspect_cols = object_columns_df.columns.drop(inspect_cols)
for i in inspect_cols:
    print(data[i].value_counts())
    print('\n ----------- \n')
print('\nRemaining Columns')
print(non_inspect_cols)

RENT        18113
MORTGAGE    16687
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64

 ----------- 

Not Verified       16283
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64

 ----------- 

10+ years    8545
< 1 year     4514
2 years      4303
3 years      4022
4 years      3353
5 years      3203
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64

 ----------- 

 36 months    28236
 60 months     9441
Name: term, dtype: int64

 ----------- 

CA    6778
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
AL     420
LA     420
KY     311
OK     285
UT     249
KS     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH    

<p>There are 9 columns formatted as string data. <br>
   2 can be converted to numeric data by dropping the percent sign (int_rate and revol_util). <br>
   1 can be converted to numeric data by mapping its string values to numbers (emp_length) <br>
   2 can be dropped as they most likely are not useful (addr_state and title) <br>
   The above steps are executed in the cell below <br>
   The remaining 4 columns seem useful and can be one-hot encoded into useable columns for making predictions. This step will be executed in the code cell after the one below. </p>

In [342]:
##This cell t

mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

data[['emp_length']] = data[['emp_length']].replace(mapping_dict)
data = data.drop(['addr_state','title'],axis=1)
data['int_rate'] = data['int_rate'].str.replace('%','').astype(float)
data['revol_util'] = data['revol_util'].str.replace('%','').astype(float)

In [343]:
# one hot encoding these columns
dummy_df = pd.get_dummies(data[['home_ownership','verification_status','purpose','term']])
#adding the one-hot encoded columns to the original dataframe
data = pd.concat([data,dummy_df],axis=1)
#dropping the original columns as they are no longer useful
data = data.drop(['home_ownership','verification_status','purpose','term'],axis=1) 

In [347]:
print(data.dtypes.value_counts())
print("We can see that there are is now only numeric data left")
data.head()

uint8      24
float64    12
int64       2
dtype: int64
We can see that there are is now only numeric data left


,loan_amnt,int_rate,installment,emp_length,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,...,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,5000.0,10.65,162.87,10,24000.0,1,27.65,0.0,1.0,3.0,...,0,0,0,0,0,0,0,0,1,0
1,2500.0,15.27,59.83,0,30000.0,0,1.00,0.0,5.0,3.0,...,0,0,0,0,0,0,0,0,0,1
2,2400.0,15.96,84.33,10,12252.0,1,8.72,0.0,2.0,2.0,...,0,0,0,0,0,1,0,0,1,0
3,10000.0,13.49,339.31,10,49200.0,1,20.00,0.0,1.0,10.0,...,0,0,0,1,0,0,0,0,1,0
5,5000.0,7.90,156.46,3,36000.0,1,11.20,0.0,3.0,9.0,...,0,0,0,0,0,0,0,1,1,0


<h1>Part 4: Making Predictions</h1>

In [349]:
def check(o,t,predictions):
    return sum((predictions == o) & (data['loan_status'] == t))
def check_all(predictions):
    tn = check(0,0,predictions)
    tp = check(1,1,predictions)
    fn = check(0,1,predictions)
    fp = check(1,0,predictions)
    fpr = fp/(fp+tn)
    tpr = tp/(tp+fn)
    print("true negatives: {}\ntrue positives: {}\nfalse negatives: {}\nfalse positives: {}".format(tn,tp,fn,fp))
    print("false positive rate: {}\ntrue positive rate: {}".format(fpr,tpr))

In [350]:
features = data.drop(['loan_status'],axis=1)
features = features.astype(np.float32)
target = data['loan_status']
scaler = preprocessing.StandardScaler().fit(features)
##target doesn't need to be scaled since it's already either 0 or 1
scaled= scaler.transform(features)

In [351]:
def timer(func):
    def wrapper(*args, **kwargs):
        t_start = time.time()

        result = func(*args, **kwargs)

        t_total = round(time.time() - t_start,2)
        print('\n{} took {}s'.format(func.__name__, t_total))
    
        return result
    return wrapper
@timer
def model_test(model):
    predictions = cross_val_predict(model,scaled,target,cv=3)
    predictions = pd.Series(predictions)
    check_all(predictions)

In [352]:
model_test(LogisticRegression(class_weight='balanced'))

true negatives: 1945
true positives: 19115
false negatives: 11399
false positives: 3121
false positive rate: 0.6160679036715357
true positive rate: 0.6264337681064429

model_test took 1.39s


In [ ]:
rf = RFECV(LogisticRegression(class_weight='balanced'))
model_test(rf)

In [ ]:
plt.figure(num=None, figsize=(8, 6), dpi=200, facecolor='w', edgecolor='k')
loan_status_first = pd.concat([data[['loan_status']],data.drop(['loan_status'],axis=1)],axis=1)
sns.heatmap(loan_status_first.corr(),xticklabels=True,yticklabels=True)
plt.show()

In [ ]:
data.columns

In [ ]:
rflr=RFECV(LogisticRegression(class_weight='balanced'))

In [ ]:
shuffled_data = data.sample(frac=1)
train = shuffled_data.iloc[0:len(data)//2]
test = shuffled_data.iloc[len(data)//2:]
train_features = train.drop(['loan_status'],axis=1)
test_features = test.drop(['loan_status'],axis=1)
train_target = train['loan_status']
test_target = test['loan_status']
scaled_train = preprocessing.StandardScaler().fit(train_features).transform(train_features)
scaled_test = preprocessing.StandardScaler().fit(train_features).transform(train_features)
rflr.fit(scaled_train,train_target)
train_predictions = rflr.predict(scaled_test)
train_false_negatives = pd.Series(((train_predictions == 0) & (train_target == 1)),name='False Negatives').astype(int)
train_features = pd.concat([train_features,train_false_negatives],axis=1)

In [ ]:
train_features = train_features.drop('False Negatives',axis=1)
rflr.fit(scaled_train,train)